In [222]:
import torch

In [223]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn

In [224]:
df=pd.read_csv('Breast-cancer.csv')

In [225]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [240]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [227]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2, random_state=42)

In [228]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)


In [229]:
encoder=LabelEncoder()
y_train=encoder.fit_transform(y_train)
y_test=encoder.transform(y_test)

In [230]:
x_train_tensor=torch.tensor(x_train)
y_train_tensor=torch.tensor(y_train)

x_test_tensor=torch.tensor(x_test)
y_test_tensor=torch.tensor(y_test)

In [241]:
class MySimpleNN(nn.Module):

    def __init__(self,num_feature):
        super().__init__()
        self.linear=nn.Linear(num_feature,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,features):
        out=self.linear(features)
        out=self.sigmoid(out)
        
        return out

In [232]:
learning_rate=0.1
epochs=25

In [233]:
loss_function=nn.BCELoss()

In [234]:
type(loss_function)

torch.nn.modules.loss.BCELoss

In [235]:
# create a model
model=MySimpleNN(x_train_tensor.shape[1])
# define loop
for i in  range(epochs):
    # forwad pass
    y_pred=model(x_train_tensor.float())

    # calculate loss
    loss=loss_function(y_pred,y_train_tensor.view(-1,1).float())

    # backward pass
    loss.backward()

    # update weights and bias
    # no_grad() is used to stop tracking the gradients
    with torch.no_grad():
        model.linear.weight-=learning_rate*model.linear.weight.grad
        model.linear.bias.data-=learning_rate*model.linear.bias.grad

        # zero the gradients
    model.linear.weight.grad.zero_()
    model.linear.bias.grad.zero_()

    # print loss in each epoch
    print(f'Epoch {i+1} loss: {loss.item()}')


Epoch 1 loss: 0.4498673677444458
Epoch 2 loss: 0.3935755491256714
Epoch 3 loss: 0.3552002012729645
Epoch 4 loss: 0.3269225060939789
Epoch 5 loss: 0.30497977137565613
Epoch 6 loss: 0.28731611371040344
Epoch 7 loss: 0.27270257472991943
Epoch 8 loss: 0.26035377383232117
Epoch 9 loss: 0.24974152445793152
Epoch 10 loss: 0.24049565196037292
Epoch 11 loss: 0.23234815895557404
Epoch 12 loss: 0.22509926557540894
Epoch 13 loss: 0.21859686076641083
Epoch 14 loss: 0.21272243559360504
Epoch 15 loss: 0.2073824405670166
Epoch 16 loss: 0.2025015503168106
Epoch 17 loss: 0.1980186104774475
Epoch 18 loss: 0.19388313591480255
Epoch 19 loss: 0.19005319476127625
Epoch 20 loss: 0.18649357557296753
Epoch 21 loss: 0.18317440152168274
Epoch 22 loss: 0.18007028102874756
Epoch 23 loss: 0.17715927958488464
Epoch 24 loss: 0.17442253232002258
Epoch 25 loss: 0.17184355854988098


In [236]:
model.linear.weight

Parameter containing:
tensor([[ 0.2989,  0.1914,  0.2359,  0.0634,  0.0773,  0.1819,  0.0346,  0.2686,
          0.2019, -0.0974,  0.2048, -0.1392,  0.1520,  0.1450,  0.0482,  0.1213,
         -0.0283,  0.0160, -0.1022, -0.0160,  0.3689,  0.3297,  0.2566,  0.3789,
          0.0829,  0.1315, -0.0168,  0.3776,  0.1940,  0.1255]],
       requires_grad=True)

In [237]:
with torch.no_grad():
    y_pred = model(x_test_tensor.float())
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == y_test_tensor.view(-1, 1).float()).float().mean()
    print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.859649121761322


# torch.optim module in pytorch 
 * The `torch.optim` module in PyTorch provides a variety of optimization algorithms used to update the parameters of your model during training. 
 * It includes common optimizers like Stochastic Gradient Descent (SGD), Adam, RMSprop, and more. It handles weight updates efficiently, including additional features like learning rate scheduling and weight decay (regularization).
 
 * The `model.parameters()` method in PyTorch retrieves an iterator over all the trainable parameters (weights and biases) in a model. These parameters are instances of `torch.nn.Parameter` and include:
 
 * - **Weights**: The weight matrices of layers like `nn.Linear`, `nn.Conv2d`, etc.
 * - **Biases**: The bias terms of layers (if they exist).
 
 * The optimizer uses these parameters to compute gradients and update them during training.
 

In [238]:


# create a model
model=MySimpleNN(x_train_tensor.shape[1])
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)
# define loop
for i in  range(epochs):
    # forwad pass
    y_pred=model(x_train_tensor.float())

    # calculate loss
    loss=loss_function(y_pred,y_train_tensor.view(-1,1).float())

    # clear the gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # update weights and bias
    optimizer.step()


    # print loss in each epoch
    print(f'Epoch {i+1} loss: {loss.item()}')


Epoch 1 loss: 0.5211790800094604
Epoch 2 loss: 0.42814135551452637
Epoch 3 loss: 0.3728615641593933
Epoch 4 loss: 0.3360072076320648
Epoch 5 loss: 0.30941566824913025
Epoch 6 loss: 0.28913792967796326
Epoch 7 loss: 0.2730361819267273
Epoch 8 loss: 0.25985103845596313
Epoch 9 loss: 0.2487911880016327
Epoch 10 loss: 0.23933342099189758
Epoch 11 loss: 0.23111754655838013
Epoch 12 loss: 0.22388708591461182
Epoch 13 loss: 0.2174539715051651
Epoch 14 loss: 0.2116771936416626
Epoch 15 loss: 0.2064485251903534
Epoch 16 loss: 0.20168355107307434
Epoch 17 loss: 0.19731523096561432
Epoch 18 loss: 0.19328954815864563
Epoch 19 loss: 0.18956255912780762
Epoch 20 loss: 0.18609783053398132
Epoch 21 loss: 0.1828652024269104
Epoch 22 loss: 0.17983907461166382
Epoch 23 loss: 0.17699791491031647
Epoch 24 loss: 0.17432311177253723
Epoch 25 loss: 0.1717987209558487


In [239]:
with torch.no_grad():
    y_pred = model(x_test_tensor.float())
    y_pred = (y_pred > 0.9).float()
    accuracy = (y_pred == y_test_tensor.view(-1, 1).float()).float().mean()
    print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.8684210777282715
